# Open Model Zoo Object Detection Demo

This demo showcases Object Detection on Open Model Zoo models with Async API. Async API usage can improve the overall frame-rate of the application, because inference and image preprocessing can occur at the same time.

This notebook allows you to select a model and an input video, as well as vary the number of streams, threads and requests for the inference.

Note: the notebook allows you to upload your own video. It is recommended to use a short video. If you use a video that is longer than a few minutes, you can adjust the `JUMP_FRAMES` setting to a larger value to increase inference speed. With the default setting every tenth frame is analyzed.

Other demo objectives are:

* Using video as input with OpenCV\*
* Visualizing the resulting bounding boxes
* Comparing results and speed of different Open Model Zoo models

See the [Python Object Detection Async Demo](../python/) for more details about the Async API, and the [Optimization Guide](https://docs.openvinotoolkit.org/latest/_docs_optimization_guide_dldt_optimization_guide.html) for more information on optimizing models.

Note that the frame rates shown in this demo are an indication and not a true measure of performance of a model. Use the [OpenVINO Benchmark Tool](https://github.com/openvinotoolkit/openvino/tree/master/inference-engine/tools/benchmark_tool) to get a better measure of performance.

This demo works with a variety of [Open Model Zoo](https://github.com/openvinotoolkit/open_model_zoo/) models and allows you to use your own video. [IPywidgets](https://github.com/jupyter-widgets/ipywidgets) widgets are used to easily select a demo video or choose a video from your PC.

**Use your own video**

Sample videos are downloaded from [Intel IoT Sample Videos](https://github.com/intel-iot-devkit/sample-videos). The link shows previews of the videos. You can optionally choose to upload your own video. If you do so, it is recommended to use a short video. To upload your own video, copy or upload an *.avi* or *.mp4* file to the *videos* subdirectory. This subdirectory will be created automatically if you run this notebook for the first time. You can also create it manually, as a subdirectory of the directory that contains this notebook. You can either copy the files to this directory manually, or use Jupyter Notebook's or Jupyter Lab's functionality to upload files. See this [short video](https://www.youtube.com/watch?v=1bd2QHqQSH4) on how to upload files in Jupyter Lab. After uploading the video, it will appear in the video selection widget. To update this widget with a newly uploaded video, either restart the notebook (Kernel menu->*Restart Kernel and Run All Cells*) or run the *Create widget* cells again.

## Imports

In [ ]:
import glob
import json
import os.path
import random
import re
import subprocess
import sys
from pathlib import Path
from time import perf_counter

import cv2
import ipywidgets as widgets
import matplotlib.pyplot as plt
from IPython.display import HTML, clear_output
from ipywidgets import Layout, interact_manual
from openvino.inference_engine import IECore

from detection_utils import ColorPalette, download_video, get_model, put_highlighted_text

# Add the Open Model Zoo common python folder to the path, to import the pipelines module.
open_model_zoo_path = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(os.curdir))))
sys.path.append(os.path.join(open_model_zoo_path, "demos", "common", "python"))

from pipelines import AsyncPipeline

## Settings

Set the file and directory paths. The default settings expect that the models are located in `open_model_zoo_models` in your `$HOME` directory, typically `C:\Users\username` or `/home/username`. You can change this by setting the `base_model_dir` variable to another directory. Set `models_file` to `models-all.lst` to use all supported models, instead of a subset. This wil increase model download and conversion time.

In [ ]:
## File settings
# Directory that contains the Open Model Zoo models. It has subdirectories "intel" and "public"
base_model_dir = os.path.expanduser("~/open_model_zoo_models")
# Directory for Open Model Zoo cache files. Caching speeds up subsequent downloads.
omz_cache_dir = os.path.expanduser("~/open_model_zoo_cache")
models_file = "models-subset.lst"  # models-subset.lst contains a subset of supported models.
# models_file = 'models.lst'  # models.lst contains all supported models.
video_directory = "videos"

## Model settings
DEVICE = "CPU"
PRECISION = "FP16"

## Demo settings
DOWNLOAD_MODELS = True  # Use Model Downloader to download models from Open Model Zoo
JUMP_FRAMES = 10  # Read every n-th frame of the input video
PROB_THRESHOLD = 0.5  # The probability threshold for detection predictions
DEFAULT_NUM_THREADS = 3  # Default number of threads to use for inference
DEFAULT_NUM_STREAMS = 3  # Default number of streams to use for inference
DEFAULT_NUM_REQUESTS = 4  # Default maximum number of requests to use for inference

## Visualization settings
PALETTE = ColorPalette(100)
FONT_SCALE = 1
THICKNESS = 2

# The settings below are only required if you want to use the Model Converter to convert models to OpenVINO IR format.
# You can use this demo with models that are already downloaded in IR format, so use of the model optimizer is optional.

# The path to the Model Optimizer is required if models need to be converted to IR. The paths below should work for default installations of
# the Intel Distribution of OpenVINO Toolkit https://software.intel.com/content/www/us/en/develop/tools/openvino-toolkit/download.html
# Adjust them if you installed OpenVINO in a different location.
# Note that you also need to install the Model Optimizer prerequisites. See the documentation for your OS at
# https://docs.openvinotoolkit.org/latest/installation_guides.html

CONVERT_MODELS = False  # Set to True to use public Open Model Zoo models and convert them with the Model Optimizer.
if CONVERT_MODELS:
    if sys.platform.startswith("win"):  # Windows
        model_optimizer_path = r"C:\Program Files (x86)\intel\openvino_2021\deployment_tools\model_optimizer\mo.py"
    else:  # Linux/MacOS
        model_optimizer_path = "/opt/intel/openvino_2021/deployment_tools/model_optimizer/mo.py"

## Download Models and convert them to IR format

The [Model Downloader](https://github.com/openvinotoolkit/open_model_zoo/blob/master/tools/downloader/README.md) downloads models from the Open Model Zoo. Models that are not in OpenVINO IR format are converted to this format by the Model Converter. 

A subset of [Open Model Zoo](https://github.com/openvinotoolkit/open_model_zoo/) models that are compatible with this demo are listed in the file `models_file` (default "models.lst") in the same folder as this notebook. By default these models are downloaded, with the `--list=models_file` argument for the Model Downloader. You can choose to download a specific model by using `--name=model_name` instead of `--list=models.lst`. If you already have downloaded Open Zoo Models, you can set the `base_model_dir` variable in the *Settings* cell to the folder that contains your models (this should be a folder with subfolders `intel` and `public`) and set `DOWNLOAD_MODELS` to `False`.

<div class="alert alert-info" style="color:black"><i>
<b>Note: </b>It will take a while to download and convert all the models. </div> 

In [ ]:
if DOWNLOAD_MODELS:
    downloader_command = os.path.join(open_model_zoo_path, "tools", "downloader", "downloader.py")
    download_result = subprocess.run(
        [
            sys.executable,  # the path to the Python executable
            downloader_command,
            "--output_dir",
            base_model_dir,
            "--cache_dir",
            omz_cache_dir,
            "--precision",
            PRECISION,
            "--list",
            models_file,
            "--jobs",
            "4",
        ],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        shell=False,
        universal_newlines=True,
    )

In [ ]:
if download_result.returncode == 0:
    print(
        "Downloading models succeeded. You can set `DOWNLOAD_MODELS=False` to save some time when you run this notebook again."
    )
else:
    print(f"Downloading models failed. The error message is: {download_result.stderr}")

In [ ]:
# Convert the models that are not in IR format to IR
if CONVERT_MODELS:
    converter_command = os.path.join(open_model_zoo_path, "tools", "downloader", "converter.py")
    converter_result = subprocess.run(
        [
            sys.executable,
            converter_command,
            "--download_dir",
            base_model_dir,
            "--list",
            models_file,
            "--precisions",
            PRECISION,
            "--mo",
            model_optimizer_path,
            "--jobs",
            "4",
        ],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        shell=False,
        universal_newlines=True,
    )
    if converter_result.returncode == 0:
        print(
            "Converting models succeeded. Set CONVERT_MODELS to False to save time when you run this notebook again."
        )
    else:
        error_message = "Not all models were converted succesfully. Check `converter_result.stderr` for more details. "
        if "No module named" in converter_result.stderr:
            error_message += "Make sure that the Model Optimizer requirements are installed before running the Model Converter. See the <a href='https://docs.openvinotoolkit.org/latest/installation_guides.html'>OpenVINO installation guide</a> for more information"
        display(HTML(f'<div class="alert alert-warning" style="color:black"><i><b>Warning: </b>{error_message}</div>'))

### Get model info

The Info Dumper returns information for the Open Model Zoo models. It returns a list of dictionaries with the model name, description, framework, license url, precisions, task type, and the subdirectory for the downloaded model.

In [ ]:
info_command = os.path.join(open_model_zoo_path, "tools", "downloader", "info_dumper.py")
info_result = subprocess.run(
    [
        sys.executable,
        info_command,
        "--list",
        models_file,
    ],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    shell=False,
    universal_newlines=True,
)
info = json.loads(info_result.stdout)

Make a list of models that will be shown as options. By default, only Intel models will be shown. Change this by uncommenting the second line to use all models. You need to make sure that the models are in IR format. This can be done by setting `CONVERT_MODELS` to `True` and running all cells. 

In [ ]:
model_names = [model["name"] for model in info if "intel" in model["subdirectory"]]
# model_names = [model["name"] for model in info]

In [ ]:
# Show an example of the information that the Info Dumper returns
info[0]

The `models_file` file, by default "models.lst", lists models that are supported by this demo, sorted by architecture. The model names can contain wildcard. For example, `face-detection-????` means that the demo supports all models with a name that starts with `face-detection-` followed by four digits. 

We create a `model_architectures` dictionary that maps the model names given by the Info Dumper, to an architecture given by `models_file`.

In [ ]:
model_architectures = {}
modellist = open(models_file).read().splitlines()

for line in modellist[1:]:
    if line.startswith("# For"):
        _, architecture = line.split("=")
    else:
        model_architectures[line] = architecture
        for modelname in model_names:
            modelpattern = re.search(line.replace("?", "[0-9]"), modelname)
            if modelpattern:
                model_architectures[modelpattern.group(0)] = architecture

## Create inference functions

Create functions that perform the inference, and display the results. 

`draw_detections` is a function that draws detection boxes found by the network on the video frame.

In [ ]:
def draw_detections(frame, detections, palette, labels, threshold, draw_landmarks=False):
    """
    Draw detection boxes on `frame`.
    """
    size = frame.shape[:2]
    for detection in detections:
        if detection.score > threshold:
            xmin = max(int(detection.xmin), 0)
            ymin = max(int(detection.ymin), 0)
            xmax = min(int(detection.xmax), size[1])
            ymax = min(int(detection.ymax), size[0])
            class_id = int(detection.id)
            color = palette[class_id]
            det_label = labels[class_id] if labels and len(labels) >= class_id else "#{}".format(class_id)
            cv2.rectangle(frame, (xmin, ymin), (xmax, ymax), color, 2)
            cv2.putText(
                frame,
                "{} {:.1%}".format(det_label, detection.score),
                (xmin, ymin - 7),
                cv2.FONT_HERSHEY_COMPLEX,
                0.6,
                color,
                1,
            )
            if draw_landmarks:
                for landmark in detection.landmarks:
                    cv2.circle(frame, landmark, 2, (0, 255, 255), 2)
    return frame

 The `do_inference_on_video` function performs the inference of a model on a specific video. The helper function `process_results` add the time to the result from the pipeline, so that the inference speed can be computed. The function opens the video file given by `input_filename` with OpenCV's `VideoCapture`. It reads the frames sequentially, `jump_frames` frames at a time. If `jump_frames=1` all frames will be read. By default `jump_frames=10` which means that every tenth frame will be read. While there are new frames, the code:

* Checks if there are results from the pipeline. If there are, it records the time, and adds the result to the list of results
* Checks if the pipeline is ready. If it is, it sees if there is a new frame. 
  * If there is a new frame (we have not reached the end of the video), the frame is read, and sent to the detector pipeline for inference. 
  * If there are no more frames, the video is closed

At the end of the function, we wait until the detector is finished, and add the final results to the list of results

In [ ]:
def do_inference_on_video(detector_pipeline, input_filename, jump_frames):
    """
    Perform asynchronous inference on a given detector_pipeline with video from input_filename.
    `jump_frames` determines how many frames will be read from the video. If jump_frames=N, every Nth frame
    of the video will be read.
    """
    resultlist = []
    next_frame_id_to_show = 0
    overall_start_time = perf_counter()
    sequential_frame_nr = 0

    def process_results(results):
        """Helper function to add inference time to results"""
        outputs, meta = results
        meta["end_time"] = perf_counter()
        meta["overall_start_time"] = overall_start_time
        return outputs, meta

    cap = cv2.VideoCapture(input_filename)

    while cap.isOpened():
        if detector_pipeline.callback_exceptions:
            raise detector_pipeline.callback_exceptions[0]

        # Process all completed requests
        results = detector_pipeline.get_result(next_frame_id_to_show)
        if results:
            resultlist.append(process_results(results))
            next_frame_id_to_show += jump_frames

        if detector_pipeline.is_ready():
            # Get new image/frame
            start_time = perf_counter()
            ret, frame = cap.read()
            sequential_frame_nr += 1
            if not ret:
                cap.release()
                continue

            # Submit for inference
            if sequential_frame_nr % jump_frames == 0:
                detector_pipeline.submit_data(frame, sequential_frame_nr, {"frame": frame, "start_time": start_time})

        else:
            # Wait for empty request
            detector_pipeline.await_any()

    detector_pipeline.await_all()

    while detector_pipeline.has_completed_request():
        next_frame_id_to_show = next(iter(detector_pipeline.completed_request_results.keys()))
        results = detector_pipeline.get_result(next_frame_id_to_show)
        if results:
            resultlist.append(process_results(results))

    return resultlist

The `make_result_frames` function takes the output of the `do_inference_on_video` function and returns a list of videoframes with detection boxes drawn on the frame.

In [ ]:
def make_result_frames(inference_result, has_landmarks):
    """ "
    Draw boxes on frames from inference results and return the list of frames.
    """
    framelist = list()

    for i, (objects, meta) in enumerate(inference_result):
        start_time = meta["start_time"]
        overall_start_time = meta["overall_start_time"]
        end_time = meta["end_time"]
        latency = (end_time - start_time) * 1000
        fps = (i + 1) / (end_time - overall_start_time)

        frame = meta["frame"]
        frame = draw_detections(
            frame=cv2.cvtColor(frame, cv2.COLOR_BGR2RGB),
            detections=objects,
            palette=PALETTE,
            labels=None,
            threshold=PROB_THRESHOLD,
            draw_landmarks=has_landmarks,
        )
        put_highlighted_text(
            frame=frame,
            message="Latency: {:.1f} ms".format(latency),
            position=(20, 30),
            font_face=cv2.FONT_HERSHEY_COMPLEX,
            font_scale=FONT_SCALE,
            color=PALETTE[0],
            thickness=THICKNESS,
        )
        put_highlighted_text(
            frame=frame,
            message="FPS: {:.1f}".format(fps),
            position=(20, 60),
            font_face=cv2.FONT_HERSHEY_COMPLEX,
            font_scale=FONT_SCALE,
            color=PALETTE[0],
            thickness=THICKNESS,
        )

        framelist.append(frame)
    return framelist

`get_results_for_model` ties everything together. It creates a pipeline for the specified model, runs inference, and creates a numpy array with results. It returns the resulting array and the FPS for inference

In [ ]:
def get_results_for_model(modelname, input_filename, num_threads, num_streams, num_requests):
    """
    Creates a pipeline for the specified model, runs inference, and creates a numpy array with results.
    The function uses the `info` and `model_architectures` dictionaries that are created in previous notebook cells.

    :param modelname: name of the model to use for inference, as given by Info Dumper
    :param input_filename: input filename for video to run inference on
    :param num_threads: number of threads for inference
    :param num_streams: number of streams for inference
    :param num_requests: maximum number of requests for inference
    :return: list of frames with drawn detection results and the inference FPS
    """
    # Create IE model
    model_info = [item for item in info if item["name"] == modelname][0]
    model_xml = os.path.join(base_model_dir, model_info["subdirectory"], PRECISION, modelname + ".xml")
    architecture_type = model_architectures[modelname]
    ie = IECore()
    model = get_model(ie=ie, model=Path(model_xml), architecture_type=architecture_type, labels=None)

    # Create Async pipeline
    plugin_config = {
        "CPU_THREADS_NUM": f"{num_threads}",
        "CPU_THROUGHPUT_STREAMS": f"{num_streams}",
    }
    detector_pipeline = AsyncPipeline(ie, model, plugin_config, device=DEVICE, max_num_requests=num_requests)

    # Do inference
    print(
        f"Starting inference. Model: {modelname}, video: {input_filename},  threads: {num_threads}, streams: {num_streams}, max_num_requests: {num_requests}"
    )
    start_time = perf_counter()
    inference_result = do_inference_on_video(
        detector_pipeline=detector_pipeline, input_filename=input_filename, jump_frames=JUMP_FRAMES
    )
    end_time = perf_counter()

    # Draw inference results on video frames and compute FPS
    has_landmarks = architecture_type == "retina"
    result_frames = make_result_frames(inference_result=inference_result, has_landmarks=has_landmarks)
    fps = len(result_frames) / (end_time - start_time)
    return result_frames, fps

## Create widget for selecting videos

Create a dropdown widget to select either a sample video that will be downloaded if needed, or an uploaded video. See the top of this notebook for information on how to upload a video of your own. Run this cell after uploading a video, to refresh the widget. 

In [ ]:
sample_video_base_url = "https://github.com/intel-iot-devkit/sample-videos/raw/master"
sample_video_filenames = open("sample_videos.lst").read().splitlines()
all_videos = glob.glob(os.path.join(video_directory, '*mp4')) + glob.glob(os.path.join(video_directory, '*avi'))
own_video_filenames = [os.path.basename(video) for video in all_videos if os.path.basename(video) not in sample_video_filenames]

video_list = [(fn[:-4], fn) for fn in own_video_filenames]
video_list += [(f"sample: {fn[:-4]}", f"{sample_video_base_url}/{fn}") for fn in sample_video_filenames]

default_video = [item for item in video_list if item[0]  == 'sample: face-demographics-walking-and-pause'][0]
default_index = video_list.index(default_video)

In [ ]:
video_index = 0 if len(own_video_filenames) > 0 else default_index  # If we uploaded a video of our own, use this video by default, otherwise use the face demographics video
video = widgets.Dropdown(options=video_list, index=video_index)

---

## Detection results of one model, drawn on video

Select a model and set the number of threads, streams and the maximum number of requests. ipywidgets\* is used to make widgets for the model and option selection. If a sample video is selected for the first time, it is downloaded to the videos directory.

In [ ]:
interact_inference = interact_manual.options(manual_name="Do inference")


@interact_inference(num_threads=(0, 8), num_streams=(0, 8), num_requests=(0, 10))
def show_results_on_model(
    model=model_names,
    video=video,
    num_threads=DEFAULT_NUM_THREADS,
    num_streams=DEFAULT_NUM_STREAMS,
    num_requests=DEFAULT_NUM_REQUESTS,
):
    """
    Perform inference and display results for the selected model, with specified number of threads, streams and max number of requests.
    NOTE: there is no error checking. Make sure that the selected model exists in IR format.
    """
    input_filename = os.path.join(video_directory, os.path.basename(video))
    if not os.path.exists(input_filename):
        download_video(video, video_directory)
    resultvideo, fps = get_results_for_model(model, input_filename, num_threads, num_streams, num_requests)

    for item in resultvideo:
        clear_output(wait=True)
        plt.imshow(item)
        plt.axis("off")
        plt.show()
    print(
        f"Finished inference. Model: {model},  threads: {num_threads}, streams: {num_streams}, max_num_requests: {num_requests}. FPS: {fps:.2f}"
    )

---

## Detection results of multiple models

Perform inference on selected models and show results on three random frames.

In [ ]:
# Create a widget to select multiple models. By default three face detection models are selected.
select_model_widget = widgets.SelectMultiple(
    description="Models",
    options=model_names,
    index=[2, 5, 7],
    layout=Layout(display="flex", flex_flow="column"),
    disabled=False,
)

In [ ]:
@interact_inference(modelnames=select_model_widget, num_threads=(0, 8), num_streams=(0, 8), num_requests=(0, 10))
def show_inference_multiple_models(
    modelnames,
    video=video,
    num_threads=DEFAULT_NUM_THREADS,
    num_streams=DEFAULT_NUM_STREAMS,
    num_requests=DEFAULT_NUM_REQUESTS,
):
    """
    Perform inference for selected models and show results on three random frames of the input video.
    """
    inference_results_multiple_models = []
    input_filename = os.path.join(video_directory, os.path.basename(video))
    if not os.path.exists(input_filename):
        download_video(video, video_directory)
    
    for i, modelname in enumerate(modelnames):
        resultvideo, fps = get_results_for_model(modelname, input_filename, num_threads, num_streams, num_requests)
        inference_results_multiple_models.append(resultvideo)
        print(f"--- Finished: FPS: {fps:.2f}")

    fig, ax = plt.subplots(3, len(inference_results_multiple_models), figsize=(18, 12), squeeze=False)
    indices = random.choices(range(len(inference_results_multiple_models[0])), k=3)
    for i, resultvideo in enumerate(inference_results_multiple_models):
        modelname = select_model_widget.value[i]
        ax[0, i].set_title(modelname)
        for j, framenr in enumerate(indices):
            ax[j, i].imshow(resultvideo[framenr])